In [ ]:
import numpy as np
import pandas as pd

import os

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import Choropleth
from folium.plugins import HeatMap
import datetime

np.random.seed(0)

In [ ]:
data = pd.read_csv("df.csv")

missing_values_columns = [col for col in data.columns
                     if data[col].isnull().any()]
data = data.drop(missing_values_columns, axis=1)

In [ ]:
tectonic_plates = pd.read_csv("datasets/all.csv" )

In [ ]:
lengths = data["Date"].str.len()
lengths.value_counts()

wrongdates = np.where([lengths == 24])[1]
data.loc[3378, "Date"] = "02/23/1975"
data.loc[7512, "Date"] = "04/28/1985"
data.loc[20650, "Date"] = "03/13/2011"
data.loc[3378, "Time"] = "02:58:41"
data.loc[7512, "Time"] = "02:53:41"
data.loc[20650, "Time"] = "02:23:34"
data['Date']= pd.to_datetime(data["Date"])

lengths = data["Time"].str.len()
lengths.value_counts()
data['Time']= pd.to_timedelta(data['Time'])

In [ ]:
data["Date_Time"]=data["Date"] +data["Time"]
data["Days"]= data.Date.dt.strftime("%A")

In [ ]:
plt.figure(figsize=(5, 3))
ts = sns.lineplot(x=data["Date"].dt.year, y="Magnitude", data=data, color="#732e75")
ts.set_title("Earthquake Time Series", color="#18508d")
ts.set_ylabel("Magnitude", color="#58508d")
ts.set_xlabel("Date", color="#58508d")

In [ ]:
colours = ["#732e75","#bc5090","#ff6361","#003f5c"]
depth = data["Depth"].values
mean_depth= data["Depth"].mean()

In [ ]:
tectonic = folium.Map(tiles="cartodbpositron", zoom_start=5)

plates = list(tectonic_plates["plate"].unique())
for plate in plates:
    plate_vals = tectonic_plates[tectonic_plates["plate"] == plate]
    lats = plate_vals["lat"].values
    lons = plate_vals["lon"].values
    points = list(zip(lats, lons))
    indexes = [None] + [i + 1 for i, x in enumerate(points) if i < len(points) - 1 and abs(x[1] - points[i + 1][1]) > 300] + [None]
    for i in range(len(indexes) - 1):
        folium.vector_layers.PolyLine(points[indexes[i]:indexes[i+1]], popup=plate, color="#58508d", fill=False, ).add_to(tectonic)

In [ ]:
from branca.element import Figure
fig = Figure(width=600, height=400)

tectonic_quake = folium.Map(tiles="cartodbpositron", zoom_start=5)
gradient = {.33: "#58508d", .66: "#ef5675", 1: "#ffa600"}
plates = list(tectonic_plates["plate"].unique())
for plate in plates:
    plate_vals = tectonic_plates[tectonic_plates["plate"] == plate]
    lats = plate_vals["lat"].values
    lons = plate_vals["lon"].values
    points = list(zip(lats, lons))
    indexes = [None] + [i + 1 for i, x in enumerate(points) if i < len(points) - 1 and abs(x[1] - points[i + 1][1]) > 300] + [None]
    for i in range(len(indexes) - 1):
        folium.vector_layers.PolyLine(points[indexes[i]:indexes[i+1]], popup=plate, fill=False,  color="#58508d").add_to(tectonic_quake)
        HeatMap(data=data[["Latitude", "Longitude"]],hue="Magnitude",min_opacity=0.5,radius=1,gradient=gradient).add_to(tectonic_quake)

fig.add_child(tectonic_quake)
fig

In [ ]:
import folium
from folium.plugins import HeatMap
import math

def haversine_distance(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
    lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    earth_radius_km = 6371
    distance = earth_radius_km * c
    return distance

your_given_latitude = 12.34
your_given_longitude = 56.78
given_coordinate = (your_given_latitude, your_given_longitude)

tectonic_quake = folium.Map(tiles="cartodbpositron", zoom_start=5)
gradient = {.33: "#58508d", .66: "#ef5675", 1: "#ffa600"}
plates = list(tectonic_plates["plate"].unique())

closest_fault_lines = []
closest_earthquakes = []
distance_to_fault_lines = float('inf')
num_closest_lines = 4

for plate in plates:
    plate_vals = tectonic_plates[tectonic_plates["plate"] == plate]
    lats = plate_vals["lat"].values
    lons = plate_vals["lon"].values
    points = list(zip(lats, lons))
    indexes = [None] + [i + 1 for i, x in enumerate(points) if i < len(points) - 1 and abs(x[1] - points[i + 1][1]) > 300] + [None]

    for i in range(len(indexes) - 1):
        folium.vector_layers.PolyLine(points[indexes[i]:indexes[i+1]], popup=plate, fill=False,  color="#58508d").add_to(tectonic_quake)
        HeatMap(data=data[["Latitude", "Longitude"]],hue="Magnitude",min_opacity=0.5,radius=1,gradient=gradient).add_to(tectonic_quake)

        for quake_lat, quake_lon in points[indexes[i]:indexes[i+1]]:
            distance = haversine_distance(given_coordinate, (quake_lat, quake_lon))
            if distance < distance_to_fault_lines:
                distance_to_fault_lines = distance
                closest_earthquakes = [(quake_lat, quake_lon, distance)]
            elif distance == distance_to_fault_lines:
                closest_earthquakes.append((quake_lat, quake_lon, distance))

        distance_to_plate = haversine_distance(given_coordinate, (plate_vals['lat'].iloc[0], plate_vals['lon'].iloc[0]))
        closest_fault_lines.append((plate, distance_to_plate))

closest_fault_lines.sort(key=lambda x: x[1])
closest_fault_lines = closest_fault_lines[:num_closest_lines]
closest_earthquakes.sort(key=lambda x: x[2])
closest_earthquakes = closest_earthquakes[:num_closest_lines]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

your_given_latitude = 12.34
your_given_longitude = 56.78
given_coordinate = (your_given_latitude, your_given_longitude)

def haversine_distance(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
    lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    earth_radius_km = 6371
    distance = earth_radius_km * c
    return distance

def find_closest_faults_quakes(row):
    given_coordinate = (row['Latitude'], row['Longitude'])
    closest_fault_lines = []
    closest_earthquakes = []

    for plate in plates:
        plate_vals = tectonic_plates[tectonic_plates["plate"] == plate]
        distance_to_plate = haversine_distance(given_coordinate, (plate_vals['lat'].iloc[0], plate_vals['lon'].iloc[0]))
        closest_fault_lines.append((plate, distance_to_plate))

    closest_fault_lines.sort(key=lambda x: x[1])
    closest_fault_lines = closest_fault_lines[:num_closest_lines]

    for index, row in data.iterrows():
        distance = haversine_distance(given_coordinate, (row['Latitude'], row['Longitude']))
        closest_earthquakes.append((row['Latitude'], row['Longitude'], distance))

    closest_earthquakes.sort(key=lambda x: x[2])
    closest_earthquakes = closest_earthquakes[:num_closest_lines]

    return closest_fault_lines, closest_earthquakes

closest_fault_lines_list = []
closest_earthquakes_list = []

for index, row in data.head(20).iterrows():
    closest_fault_lines, closest_earthquakes = find_closest_faults_quakes(row)
    closest_fault_lines_list.append(closest_fault_lines)
    closest_earthquakes_list.append(closest_earthquakes)

results_df = pd.DataFrame({
    'Latitude': data.head(20)['Latitude'],
    'Longitude': data.head(20)['Longitude'],
    'Closest_Fault_Lines': closest_fault_lines_list,
    'Closest_Earthquakes': closest_earthquakes_list
})

print(results_df)

In [ ]:
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

data = data.sort_values(by="Magnitude", ascending=False)

def haversine_distance(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
    lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    earth_radius_km = 6371
    distance = earth_radius_km * c
    return distance

distances_to_fault = []
magnitudes_list = []
depths_list = []

for index, quake_row in tqdm(data.head(500).iterrows(), total=500):
    min_distance_to_fault = float('inf')
    for plate in plates:
        plate_vals = tectonic_plates[tectonic_plates["plate"] == plate]
        lats = plate_vals["lat"].values
        lons = plate_vals["lon"].values
        points = list(zip(lats, lons))
        for i in range(len(points) - 1):
            distance_to_fault = haversine_distance((quake_row["Latitude"], quake_row["Longitude"]), points[i])
            if distance_to_fault < min_distance_to_fault:
                min_distance_to_fault = distance_to_fault

        distances_to_fault.append(min_distance_to_fault)
        magnitudes_list.append(quake_row["Magnitude"])
        depths_list.append(quake_row["Depth"])

In [ ]:
plt.figure(figsize=[5,3])
plt.scatter(distances_to_fault, magnitudes_list, alpha=0.7)
plt.ylabel("Distance to Fault (km)")
plt.xlabel("Magnitude")
plt.grid(True)
plt.xlim([200, 700])
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from scipy.optimize import curve_fit

# Assuming you have already imported the necessary libraries and data

model_data = pd.DataFrame({"Distance_to_Fault": distances_to_fault,
                           "Depth": depths_list,
                           "Magnitude": magnitudes_list})

X = model_data["Distance_to_Fault"]
y_depth = model_data["Depth"]
y_magnitude = model_data["Magnitude"]

def negative_exponential_func(x, a, b, c):
    return a * np.exp(-b * x) + c

depth_params, depth_covariance = curve_fit(negative_exponential_func, X, y_depth)
magnitude_params, magnitude_covariance = curve_fit(negative_exponential_func, X, y_magnitude)

plt.figure(figsize=[5, 3])
plt.scatter(distances_to_fault, depths_list, label="Depth", alpha=0.7)
plt.scatter(distances_to_fault, magnitudes_list, label="Magnitude", alpha=0.7)
plt.plot(distances_to_fault, negative_exponential_func(X, *depth_params), color='red', label='Depth Exponential Fit')
plt.plot(distances_to_fault, negative_exponential_func(X, *magnitude_params), color='blue', label='Magnitude Exponential Fit')
plt.xlabel("Distance to Fault (km)")
plt.ylabel("Depth / Magnitude")
plt.title("Depth and Magnitude based on Distance to Fault (<= 200 km)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model_data = pd.DataFrame({"Distance_to_Fault": distances_to_fault,
                           "Depth": depths_list,
                           "Magnitude": magnitudes_list})

X = model_data[["Distance_to_Fault"]]
y_depth = model_data["Depth"]
y_magnitude = model_data["Magnitude"]
depth_model = LinearRegression()
magnitude_model = LinearRegression()

depth_model.fit(X, y_depth)
magnitude_model.fit(X, y_magnitude)

In [ ]:
from scipy.spatial import KDTree

tree = KDTree(data[["Latitude", "Longitude"]])

def find_nearest_earthquakes(coord, k=5):
    distance_to_nearest, index_of_nearest = tree.query(coord, k=k)
    return data.iloc[index_of_nearest]

coordinate_to_search = (37, 37)
nearest_earthquakes = find_nearest_earthquakes(coordinate_to_search)

nearest_earthquakes.head()

In [ ]:
from scipy.spatial import KDTree

tree = KDTree(data[["Latitude", "Longitude"]])

fault_line_tree = KDTree(tectonic_plates[["lat", "lon"]])

def find_nearest_fault_line(coord):
    distance_to_nearest_fault, index_of_nearest_fault = fault_line_tree.query(coord)
    nearest_fault_line = tectonic_plates.iloc[index_of_nearest_fault]
    return nearest_fault_line

def calculate_distance_to_fault(row):
    earthquake_coordinate = (row["Latitude"], row["Longitude"])
    nearest_fault_line = find_nearest_fault_line(earthquake_coordinate)
    distance_to_fault = haversine_distance(earthquake_coordinate, (nearest_fault_line["lat"], nearest_fault_line["lon"]))
    return distance_to_fault

data["Distance_to_Fault"] = data.apply(calculate_distance_to_fault, axis=1)

magnitude_weight = 0.33
depth_weight = 0.33
distance_to_fault_weight = 0.33

def get_danger_score(earthquake):
    magnitude = earthquake["Magnitude"]
    depth = earthquake["Depth"]
    distance_to_fault = earthquake["Distance_to_Fault"]
    log_distance_to_fault = np.log(distance_to_fault + 1)
    composite_score = (magnitude_weight * magnitude) + (depth_weight * depth) + (distance_to_fault_weight * log_distance_to_fault)
    return composite_score

In [ ]:
data["Composite Score"] = data.apply(get_danger_score, axis=1)
subset = data[["Date", "Latitude", "Longitude", "Magnitude", "Depth", "Distance_to_Fault", "Composite Score"]]
subset = subset.sort_values(by="Composite Score", ascending=False)

In [ ]:
demographics = pd.read_csv("datasets/socioeconomic.csv", encoding="ISO-8859-1")

missing_values_columns_demographics = [col for col in demographics.columns if demographics[col].isnull().any()]
demographics = demographics.drop(missing_values_columns_demographics, axis=1)

In [ ]:
demographics_2010 = demographics[demographics["year"] == 2010]

plt.figure(figsize=(5, 3))
plt.scatter(demographics_2010["SES"], demographics_2010["gdppc"], alpha=0.7)
plt.xlabel("Socioeconomic Score (SES)")
plt.ylabel("GDP per Capita (gpppc)")
plt.title("SES vs. GDP per Capita in 2010")
plt.grid(True)
plt.show()

In [ ]:
!pip install reverse_geocode
import reverse_geocode

In [ ]:
tqdm.pandas()

def get_ses_gdppc_popshare(row):
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    location = reverse_geocode.search([(latitude, longitude)])
    country = location[0]["country"]

    demographics_country_2010 = demographics[(demographics["country"] == country) & (demographics["year"] == 2010)]

    if not demographics_country_2010.empty:
        ses_value = demographics_country_2010["SES"].iloc[0]
        gdppc_value = demographics_country_2010["gdppc"].iloc[0]
        popshare_value = demographics_country_2010["popshare"].iloc[0]
        return pd.Series({"SES": ses_value, "gdppc": gdppc_value, "popshare": popshare_value})
    else:
        return pd.Series({"SES": None, "gdppc": None, "popshare": None})
subset[["SES", "gdppc", "popshare"]] = subset.progress_apply(get_ses_gdppc_popshare, axis=1)
subset.head()

In [ ]:
from scipy import stats

small = subset.groupby("gdppc")["Composite Score"].mean().reset_index()

In [ ]:
economic = pd.read_csv("datasets/economicdata.csv")

In [ ]:
latitude_input = 37
longitude_input = 37

def locate_nearest_earthquakes(coord):
    data["Distance_to_Fault"] = data.apply(calculate_distance_to_fault, axis=1)
    data["Distance_to_Input"] = data.apply(lambda row: haversine_distance(coord, (row["Latitude"], row["Longitude"])), axis=1)
    nearest_earthquakes = data.nsmallest(5, "Distance_to_Input")
    nearest_earthquakes["Weight"] = np.log(1 + nearest_earthquakes["Distance_to_Input"])
    nearest_earthquakes["Weighted_Danger_Level"] = nearest_earthquakes["Weight"] * nearest_earthquakes.apply(get_danger_score, axis=1)
    weighted_composite_score = nearest_earthquakes["Weighted_Danger_Level"].sum() / nearest_earthquakes["Weight"].sum()
    return weighted_composite_score

input_coordinate = (latitude_input, longitude_input)

weighted_composite_score = locate_nearest_earthquakes(input_coordinate)

location = reverse_geocode.search([(latitude_input, longitude_input)])
country_identified = location[0]["country"]

demographics_country_2010 = demographics[(demographics["country"] == country_identified) & (demographics["year"] == 2010)]

identified_extended = economic[economic["Countries"] == country_identified]

country_regulation = float(identified_extended["Regulation"].iloc[0])
country_regulatory_burden = float(identified_extended["Regulatory Burden"].iloc[0])
country_labor_market_regulations = float(identified_extended["Labor market regulations"].iloc[0])
country_licensing_restrictions = float(identified_extended["Licensing restrictions"].iloc[0])
country_business_regulations = float(identified_extended["Business regulations"].iloc[0])
country_administrative_requirements = float(identified_extended["Administrative requirements"].iloc[0])

if not demographics_country_2010.empty:
    ses_value = demographics_country_2010["SES"].iloc[0]
    gdppc_value = demographics_country_2010["gdppc"].iloc[0]
else:
    print(f"Data not available for {country_identified} in the year 2010.")

In [ ]:
sum = country_regulation + country_regulatory_burden + country_labor_market_regulations + country_licensing_restrictions + country_business_regulations + country_administrative_requirements
regulatory_score = round(sum/6, 3)
print(f"Regulatory Score: {regulatory_score}")
economic_score = round(ses_value/10, 3)
print(f"Economic Score: {economic_score}")
risk_score = round(weighted_composite_score, 3)
print(f"Risk Score: {risk_score}")